# 🎭 Drama Data Processor

Notebook này dùng để:
- Chuẩn hóa dữ liệu thô từ các bài viết mạng xã hội (JSON)
- Lọc bình luận spam
- Gán nhãn dư luận bằng zero-shot classification
- Tóm tắt các nhóm ý kiến chính

👉 Sử dụng mô hình `facebook/bart-large-mnli` từ HuggingFace Transformers


In [ ]:
!pip install -q transformers accelerate tqdm


In [ ]:
from transformers import pipeline
import json
import os
from datetime import datetime
from tqdm import tqdm

In [ ]:
path_post = "data_fb.json"
path_check = "links_fb.json"
path_data_out = "clean_data.json"
path_static_out = "static_data.json"

In [ ]:
import json

with open(path_post, 'r') as f:
    data_post = json.load(f)

with open(path_check, 'r') as f:
    data_check = json.load(f)

print("Số bài post:", len(data_post))
print("Số entry trong check_link:", len(data_check))

In [ ]:
# Load file check_link
with open(path_check, "r") as f:
    check_data = json.load(f)

check_time_map = {item["source_url"]: item["time"] for item in check_data}

In [ ]:
# Zero-shot model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Nhãn dư luận chính
labels = ["Đồng tình", "Chỉ trích", "Chế giễu", "Ủng hộ", "Phản đối", "Trung lập", "Thông cảm"]

# Hàm chuẩn hóa thời gian
def normalize_time(post_url, raw_time):
    try:
        return check_time_map.get(post_url, raw_time)
    except:
        return raw_time

# Load post gốc
with open(path_post, "r") as f:
    posts = json.load(f)

clean_data = []
static_data = []

for post in tqdm(posts, desc="Đang xử lý bài viết"):
    url = post.get("url", "")
    content = post.get("content", "")
    poster = post.get("poster", "")
    time = normalize_time(url, post.get("time", ""))
    comments = post.get("comments", [])

    opinion_summary = {}
    opinion_examples = {}

    for cmt in comments:
        text = cmt.get("text", "").strip()
        if not text or len(text) < 6: continue
        result = classifier(text, labels)
        best_label = result["labels"][0]
        score = result["scores"][0]

        if best_label not in opinion_summary:
            opinion_summary[best_label] = 0
            opinion_examples[best_label] = []

        opinion_summary[best_label] += 1
        if len(opinion_examples[best_label]) < 3:
            opinion_examples[best_label].append(text)

    # Cleaned data (cho LLM dùng)
    clean_post = {
        "url": url,
        "poster": poster,
        "time": time,
        "content": content,
        "summary": {
            "num_opinions": len(opinion_summary),
            "opinions": [
                {
                    "label": label,
                    "description": f"Số lượng comment: {count}",
                    "example_comments": opinion_examples[label]
                } for label, count in opinion_summary.items()
            ]
        }
    }
    clean_data.append(clean_post)

    # Static data (cho thống kê)
    static_data.append({
        "url": url,
        "content": content,
        "summary_stat": opinion_summary
    })

# Lưu file kết quả
with open(path_data_out, "w", encoding="utf-8") as f:
    json.dump(clean_data, f, ensure_ascii=False, indent=2)

with open(path_static_out, "w", encoding="utf-8") as f:
    json.dump(static_data, f, ensure_ascii=False, indent=2)

print("✅ Hoàn tất xử lý và lưu dữ liệu.")

✅ **Xử lý xong!**
- Dữ liệu đã được lưu tại `clean_data.json`
- Bạn có thể tiếp tục gộp nhiều file lại nếu cần
